# Data Understanding

In [2]:
import pandas as pd
pd.set_option('display.max_rows', 200)
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

from fuzzywuzzy import fuzz, process

## Import Data

### Corona Time Series

In [3]:
confirmed = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv")
deaths = pd.read_csv("https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv")
recovered = pd.read_csv("https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv")

In [4]:
confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,48,50,50,50,53,59,70,75,82,114
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,...,420,461,502,511,581,639,639,701,773,839
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,130,138,150,150,160,178,178,200,212,226
3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,4,...,83,93,99,117,129,149,149,197,238,428


In [5]:
deaths.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20
0,NaN,Thailand,15.0000,101.0000,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
1,NaN,Japan,36.0000,138.0000,0,0,0,0,0,0,...,6,6,6,10,10,15,16,19,22,22
2,NaN,Singapore,1.2833,103.8333,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,Nepal,28.1667,84.2500,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,Malaysia,2.5000,112.5000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
recovered.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20
0,NaN,Thailand,15.0000,101.0000,0,0,0,0,2,2,...,31,31,31,31,33,34,34,35,35,35
1,NaN,Japan,36.0000,138.0000,0,0,0,0,1,1,...,46,76,76,76,101,118,118,118,118,118
2,NaN,Singapore,1.2833,103.8333,0,0,0,0,0,0,...,78,78,78,78,78,96,96,97,105,105
3,NaN,Nepal,28.1667,84.2500,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
4,NaN,Malaysia,2.5000,112.5000,0,0,0,0,0,0,...,22,23,24,24,24,26,26,26,35,42


In [16]:
dates = confirmed.columns[4:]

conf_df_long = confirmed.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Confirmed')

deaths_df_long = deaths.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Deaths')

recv_df_long = recovered.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Recovered')

full_table = pd.concat([conf_df_long, deaths_df_long['Deaths'], recv_df_long['Recovered']], 
                       axis=1, sort=False)

full_table.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Thailand,15.0000,101.0000,1/22/20,2,0,0
1,NaN,Japan,36.0000,138.0000,1/22/20,2,0,0
2,NaN,Singapore,1.2833,103.8333,1/22/20,0,0,0
3,NaN,Nepal,28.1667,84.2500,1/22/20,0,0,0
4,NaN,Malaysia,2.5000,112.5000,1/22/20,0,0,0


In [10]:
full_table.shape

(24300, 8)

In [13]:
full_table.isna().sum()

Province/State    7506
Country/Region       0
Lat                  0
Long                 0
Date                 0
Confirmed            0
Deaths               0
Recovered            0
dtype: int64

In [19]:
full_table[full_table['Country/Region']=='China']

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
155,Hubei,China,30.9756,112.2707,1/22/20,444,17,28
159,Guangdong,China,23.3417,113.4244,1/22/20,26,0,0
160,Henan,China,33.8820,113.6140,1/22/20,5,0,0
161,Zhejiang,China,29.1832,120.0934,1/22/20,10,0,0
162,Hunan,China,27.6104,111.7088,1/22/20,4,0,0
...,...,...,...,...,...,...,...,...
24039,Inner Mongolia,China,44.0935,113.9448,3/15/20,75,1,71
24040,Ningxia,China,37.2692,106.1655,3/15/20,75,0,73
24044,Qinghai,China,35.7452,95.9956,3/15/20,18,0,18
24045,Macau,China,22.1667,113.5500,3/15/20,10,0,10


In [22]:
df = full_table.groupby(['Country/Region', 'Date'], as_index=False)[['Confirmed', 'Deaths', 'Recovered']].sum()

In [23]:
country_dict = {'Mainland China': 'China',
                'Korea, South': 'Republic of Korea',
                'US':'United States of America',
                'Taiwan*': "China, Taiwan Province of China",
                'Bolivia': 'Bolivia (Plurinational State of)',
                "Iran": "Iran (Islamic Republic of)",
                "Russia": "Russian Federation",
                "Vietnam": "Viet Nam",
                "Brunei":"Brunei Darussalam",
                "Moldova": "Republic of Moldova",
                "Cote d'Ivoire": "Côte d'Ivoire",
                "Reunion": "Réunion",
                "Congo (Kinshasa)":"Democratic Republic of the Congo",
                "occupied Palestinian territory": "State of Palestine",
                "Curacao":"Curaçao",
                "Venezuela":"Venezuela (Bolivarian Republic of)",
                #"Jersey": "United Kingdom",
                #"Guernsey": "United Kingdom"
               }
# Replacing names of countries eg. "Mainland China" with "China" 
df['Country'] = df['Country/Region'].replace(country_dict)

In [20]:
df[df['Country/Region'].str.contains("jersey", case=False)]

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude
202,NaN,Jersey,2020-03-14T16:33:03,2,0,0,49.19,-2.11


In [22]:
pop[pop.Region.str.contains("jer", case=False)]

,Region,Country code,Type,Parent code,Population


### Population

In [6]:
pop = pd.read_excel("data/pop/un_pop_clean.xlsx", index_col=0)
# pop = pop.rename(columns={'2018':'population', "Country Name":"Country"})

# Change Population from thousands to ones
pop.iloc[:,-1] = pop.Population * 1000

In [280]:
pop.head()

,Region,Country code,Type,Parent code,Population
1,WORLD,900,World,0,7.794799e+09
3,More developed regions,901,Development Group,1803,1.273304e+09
4,Less developed regions,902,Development Group,1803,6.521494e+09
5,Least developed countries,941,Development Group,902,1.057438e+09
6,"Less developed regions, excluding least develo...",934,Development Group,902,5.464056e+09


In [26]:
pop.head(25)

,Region,Country code,Type,Parent code,Population
1,WORLD,900,World,0,7.794799e+09
3,More developed regions,901,Development Group,1803,1.273304e+09
4,Less developed regions,902,Development Group,1803,6.521494e+09
5,Least developed countries,941,Development Group,902,1.057438e+09
6,"Less developed regions, excluding least develo...",934,Development Group,902,5.464056e+09
7,"Less developed regions, excluding China",948,Development Group,1803,5.050208e+09
8,Land-locked Developing Countries (LLDC),1636,Special other,1803,5.331434e+08
9,Small Island Developing States (SIDS),1637,Special other,1803,7.207610e+07
11,High-income countries,1503,Income Group,1802,1.263093e+09
12,Middle-income countries,1517,Income Group,1802,5.753052e+09


In [40]:
pop.Population[1]

7794798729.0

In [44]:
pop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 285 entries, 1 to 289
Data columns (total 5 columns):
Region          285 non-null object
Country code    285 non-null int64
Type            285 non-null object
Parent code     285 non-null int64
Population      285 non-null float64
dtypes: float64(1), int64(2), object(2)
memory usage: 23.4+ KB


In [97]:
pop[pop.Region.str.contains("Kosovo", case=False)]

,Region,Country code,Type,Parent code,Population


### Location

In [63]:
location = pd.read_excel("data/pop/un_location_clean.xlsx", index_col=0)

# Select only countries, not region
#location = location.loc[location.LocTypeName == 'Country/Area'].iloc[:, [0, 2, 3]].reset_index(drop=True)

In [64]:
location.head()

,Location,Notes,LocID,ISO3_Code,LocType,LocTypeName,ParentID,WorldID,SubRegID,SubRegName,...,WB_LMIC,WB_LIC,WB_NoIncomeGroup,MaxHIV_Male,MaxHIV_Female,MaxHIV_BothSexes,YearMaxHIV_BothSexes,HIVAIDSMortalityImpact_AgePattern,HIVAIDSMortalityImpact_e0,TotPop2019LessThan90k
Index,,,,,,,,,,,,,,,,,,,,,
1,WORLD,NaN,900,NaN,NaN,NaN,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,UN development groups,a,1803,NaN,25.0,Label/Separator,900,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,More developed regions,b,901,NaN,5.0,Development group,1803,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Less developed regions,c,902,NaN,5.0,Development group,1803,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Least developed countries,d,941,NaN,5.0,Development group,902,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
group = location[location.LocTypeName.isin(['Development group', 'Income group'])][['Location', 'LocID', 'LocTypeName']].reset_index(drop=True)
group = group.rename(columns={'Location': 'Income'})

In [66]:
location['WD'] = location.loc[:, location.columns.str.startswith("WB")].min(axis=1)

In [67]:
location = location.merge(group, how='left', left_on='WD', right_on = 'LocID', suffixes=('', 'y_'))

### Country names

In [92]:
country = pd.read_csv("data/pop/WDICountry.csv").iloc[:,:10]

# Drop regions or special areas such as OECD countries. Only keep countries
country = country.loc[~country.Region.isna()]

In [107]:
country.sample(5)

,Country Code,Short Name,Table Name,Long Name,2-alpha code,Currency Unit,Special Notes,Region,Income Group,WB-2 code
177,NZL,New Zealand,New Zealand,New Zealand,NZ,New Zealand dollar,Fiscal year end: March 31; reporting period fo...,East Asia & Pacific,High income,NZ
239,TTO,Trinidad and Tobago,Trinidad and Tobago,Republic of Trinidad and Tobago,TT,Trinidad and Tobago dollar,NaN,Latin America & Caribbean,High income,TT
65,EGY,Egypt,"Egypt, Arab Rep.",Arab Republic of Egypt,EG,Egyptian pound,NaN,Middle East & North Africa,Lower middle income,EG
9,ASM,American Samoa,American Samoa,American Samoa,AS,U.S. dollar,NaN,East Asia & Pacific,Upper middle income,AS
249,UZB,Uzbekistan,Uzbekistan,Republic of Uzbekistan,UZ,Uzbek sum,NaN,Europe & Central Asia,Lower middle income,UZ


In [94]:
country.shape

(217, 10)

In [216]:
location = location.merge(country.iloc[:, [0,1]], left_on='ISO3_Code', right_on='Country Code', how='left')
location = location.iloc[:, [0,1,2,4]]

In [217]:
location.head()

,Location,LocID,ISO3_Code,Country Code
0,Burundi,108,BDI,BDI
1,Comoros,174,COM,COM
2,Djibouti,262,DJI,DJI
3,Eritrea,232,ERI,ERI
4,Ethiopia,231,ETH,ETH


In [156]:
location.isna().sum()

Location         0
LocID            0
ISO3_Code        0
Country Code    19
Short Name      19
dtype: int64

In [162]:
location[location['Short Name'].isna()]

,Location,LocID,ISO3_Code,Short Name
9,Mayotte,175,MYT,NaN
11,Réunion,638,REU,NaN
47,Saint Helena,654,SHN,NaN
57,Western Sahara,732,ESH,NaN
93,"China, Taiwan Province of China",158,TWN,NaN
109,Anguilla,660,AIA,NaN
114,"Bonaire, Sint Eustatius and Saba",535,BES,NaN
122,Guadeloupe,312,GLP,NaN
125,Martinique,474,MTQ,NaN
126,Montserrat,500,MSR,NaN


### Merge Location and Population

In [68]:
# Merge with location data to include Country ISO3 Code
pop = pop.merge(location, left_on="Country code", right_on="LocID")
pop = pop[['Region', 'Population', 'ISO3_Code', 'Income']]

In [69]:
pop.head()

,Region,Population,ISO3_Code,Income
0,WORLD,7.794799e+09,NaN,NaN
1,More developed regions,1.273304e+09,NaN,NaN
2,Less developed regions,6.521494e+09,NaN,NaN
3,Least developed countries,1.057438e+09,NaN,NaN
4,"Less developed regions, excluding least develo...",5.464056e+09,NaN,NaN


In [71]:
pop.to_pickle('data/pop/pop.pkl')

## Match

In [ ]:
scorer_dict = { 'R':fuzz.ratio,  # Good
                'PR': fuzz.partial_ratio,  # So so
                'TSeR': fuzz.token_set_ratio,  # Good ***
                'TSoR': fuzz.token_sort_ratio,  # Good ***
                'PTSeR': fuzz.partial_token_set_ratio,  # Don't use
                'PTSoR': fuzz.partial_token_sort_ratio, # Don't use
                'WR': fuzz.WRatio,  # Don't use
                'QR': fuzz.QRatio,  # Good
                'UWR': fuzz.UWRatio,  # Don't use
                'UQR': fuzz.UQRatio }  # Good

In [73]:
%%time

def match_country(query, choices, scorer=fuzz.partial_token_sort_ratio, score_cutoff=80):

    match = process.extractOne(query=query, choices=choices, scorer=scorer, score_cutoff=score_cutoff)
    if match:
        return query, match[0], match[1], match[2]
    else:
        return (np.nan, np.nan, np.nan, np.nan)

    
result = df['Country/Region'].drop_duplicates().apply(match_country, choices=pop.Region,  
                                              scorer=fuzz.QRatio, score_cutoff=0)

result = pd.DataFrame(result.tolist(), columns=['country', 'matched_country', 'score', 'key'])

CPU times: user 365 ms, sys: 4.36 ms, total: 370 ms
Wall time: 387 ms


In [75]:
result

,country,matched_country,score,key
0,China,China,100,128
1,Panama,Panama,100,186
2,Australia,Australia,100,203
3,Croatia,Croatia,100,261
4,Burkina Faso,Burkina Faso,100,65
5,Albania,Albania,100,258
6,Canada,Canada,100,286
7,United Kingdom,United Kingdom,100,256
8,Philippines,Philippines,100,143
9,Nepal,Nepal,100,123


In [134]:
def merge_match(df, result, fonds_column="fonds_clean",ISIN_column="ISIN", result_isin="ISIN", result_fonds="fonds"):
    '''Fill some missing ISIN values with matching the fund names.'''

    # Create a dictionary from current fond-ISIN pairs
    match_dict = dict(zip(result[result_fonds], result[result_isin]))

    # Fill ISIN column with mapped values
    df[ISIN_column] = df[ISIN_column].fillna(df[fonds_column].map(match_dict))
    
    return df

# Merge the previously matched result with the df
df = preprocessing.merge_match(df, result[result.score>70])

NameError: name 'preprocessing' is not defined

## Merge

In [292]:
merged = df.merge(pop, how="left", left_on="Country", right_on="Region")
merged.drop('Region', axis=1, inplace=True)

In [300]:
merged.sample(5)

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude,Country,Population,ISO3_Code
228,NaN,Guyana,2020-03-11T20:00:00,1,1,0,5.000000,-58.750000,Guyana,7.865590e+05,GUY
205,North Dakota,US,2020-03-11T20:00:00,1,0,0,47.528912,-99.784012,United States of America,3.310026e+08,USA
27,Inner Mongolia,China,2020-03-11T03:53:03,75,1,71,44.093500,113.944800,China,1.439324e+09,CHN
26,Xinjiang,China,2020-03-11T02:18:14,76,3,73,41.112900,85.240100,China,1.439324e+09,CHN
172,Alaska,US,2020-03-11T20:00:00,1,0,0,61.370716,-152.404419,United States of America,3.310026e+08,USA


In [294]:
merged.isna().sum()

Province/State    117
Country/Region      0
Last Update         0
Confirmed           0
Deaths              0
Recovered           0
Latitude            0
Longitude           0
Country             0
Region              1
Population          1
ISO3_Code           1
dtype: int64

In [301]:
merged[merged.Population.isna()]

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude,Country,Population,ISO3_Code
60,Diamond Princess,Cruise Ship,2020-03-11T20:00:00,696,7,325,35.4498,139.6649,Cruise Ship,NaN,NaN


In [302]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 230 entries, 0 to 229
Data columns (total 11 columns):
Province/State    113 non-null object
Country/Region    230 non-null object
Last Update       230 non-null object
Confirmed         230 non-null int64
Deaths            230 non-null int64
Recovered         230 non-null int64
Latitude          230 non-null float64
Longitude         230 non-null float64
Country           230 non-null object
Population        229 non-null float64
ISO3_Code         229 non-null object
dtypes: float64(3), int64(3), object(5)
memory usage: 21.6+ KB


## Add Extra Columns

In [304]:
merged.groupby('Country/Region').sum()

,Confirmed,Deaths,Recovered,Latitude,Longitude,Population
Country/Region,,,,,,
Afghanistan,7,0,0,33.000000,65.000000,3.892834e+07
Albania,33,1,0,41.153300,20.168300,2.877800e+06
Algeria,26,2,8,28.033900,1.659600,4.385104e+07
Andorra,1,0,0,42.506300,1.521800,7.726500e+04
Antigua and Barbuda,1,0,0,17.060800,-61.796400,9.792800e+04
Argentina,31,2,0,-38.416100,-63.616700,4.519578e+07
Armenia,8,0,0,40.069100,45.038200,2.963234e+06
Aruba,2,0,0,12.521100,-69.968300,1.067660e+05
Australia,200,3,23,-220.525800,1269.500300,2.294989e+08


In [328]:
df.groupby("Country/Region", as_index=False)[['Confirmed', 'Deaths','Recovered']].sum()

,Country/Region,Confirmed,Deaths,Recovered
0,Afghanistan,7,0,0
1,Albania,33,1,0
2,Algeria,26,2,8
3,Andorra,1,0,0
4,Antigua and Barbuda,1,0,0
5,Argentina,31,2,0
6,Armenia,8,0,0
7,Aruba,2,0,0
8,Australia,200,3,23
9,Austria,504,1,6


In [321]:
merged = df.groupby("Country/Region", as_index=False)[['Confirmed', 'Deaths','Recovered']].sum()

In [322]:
merged.head()

,Country/Region,Confirmed,Deaths,Recovered
0,Afghanistan,7,0,0
1,Albania,33,1,0
2,Algeria,26,2,8
3,Andorra,1,0,0
4,Antigua and Barbuda,1,0,0


In [312]:
agg = merged.merge(pop, how="left", left_on="Country/Region", right_on="Region")

In [313]:
agg.head()

,Country/Region,Confirmed,Deaths,Recovered,Region,Population,ISO3_Code
0,Afghanistan,7,0,0,Afghanistan,38928341.0,AFG
1,Albania,33,1,0,Albania,2877800.0,ALB
2,Algeria,26,2,8,Algeria,43851043.0,DZA
3,Andorra,1,0,0,Andorra,77265.0,AND
4,Antigua and Barbuda,1,0,0,Antigua and Barbuda,97928.0,ATG


In [314]:
agg['Active'] = agg.Confirmed - (agg.Deaths + agg.Recovered)

In [316]:
agg['Confirmed_per_Cap'] = agg.Confirmed/ agg.Population
agg['Deaths_per_Cap'] = agg.Deaths/ agg.Population
agg['Recovered_per_Cap'] = agg.Recovered/ agg.Population
agg['Active_per_Cap'] = agg.Active/ agg.Population

In [317]:
agg.head()

,Country/Region,Confirmed,Deaths,Recovered,Region,Population,ISO3_Code,Active,Confirmed_per_Cap,Deaths_per_Cap,Recovered_per_Cap,Active_per_Cap
0,Afghanistan,7,0,0,Afghanistan,38928341.0,AFG,7,1.798176e-07,0.000000e+00,0.000000e+00,1.798176e-07
1,Albania,33,1,0,Albania,2877800.0,ALB,32,1.146709e-05,3.474877e-07,0.000000e+00,1.111961e-05
2,Algeria,26,2,8,Algeria,43851043.0,DZA,16,5.929163e-07,4.560895e-08,1.824358e-07,3.648716e-07
3,Andorra,1,0,0,Andorra,77265.0,AND,1,1.294247e-05,0.000000e+00,0.000000e+00,1.294247e-05
4,Antigua and Barbuda,1,0,0,Antigua and Barbuda,97928.0,ATG,1,1.021158e-05,0.000000e+00,0.000000e+00,1.021158e-05


In [68]:
agg.to_excel("corona with population.xlsx")

In [222]:
agg[agg['Country/Region'].str.contains(r"San Marino", regex=True)]

,Country/Region,Confirmed,Deaths,Recovered,Country,Country Code,population,Active,Confirmed_per_Cap,Deaths_per_Cap,Recovered_per_Cap,Active_per_Cap
92,San Marino,62,2,0,San Marino,SMR,33.785,60,1.835134,0.059198,0.0,1.775936
